In [1]:
import os
import datasets
import transformers
import peft
import trl

In [2]:
import torch
import concurrent.futures
from tqdm import tqdm

In [3]:
EFFICIENT_TRAINING = True 

In [4]:
base_model_name = "Qwen/Qwen2.5-3B-Instruct"

In [5]:
def get_tokenizer(path=None):
  tokenizer = transformers.AutoTokenizer.from_pretrained(path or base_model_name, trust_remote_code=True)
  if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|fim_pad|>'})
    tokenizer.pad_token = "<|fim_pad|>"
  return tokenizer

In [34]:
def reason_on_prompt(model, tokenizer, user_prompt):
    with torch.no_grad():
        user_prompt = user_prompt.replace("\n", "")
        
        prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + user_prompt + "<|im_end|>\n<|im_start|>assistant\n"
        stop_token_ids = tokenizer("<|im_start|><|im_end|>")["input_ids"]
        prompt += "<|im_start|>think"
        print("***Initial prompt:***\n", prompt)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        MAX_THINK_TOKENS = 1500
        o = model.generate(
          **inputs,
          max_new_tokens=MAX_THINK_TOKENS,
          eos_token_id=stop_token_ids,
          temperature=0.1,
        )
        assert len(o) == 1
        o = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, o)]

        generated_text = tokenizer.decode(o[0], skip_special_tokens=True)
        prompt += generated_text
        print("***Initial prompt with thinking:***\n", prompt)
        
        # Wait and think once more if there is space
        if len(o[0]) < MAX_THINK_TOKENS:
            ignore_str = "Wait"
            prompt += ignore_str
            max_tokens_thinking = MAX_THINK_TOKENS - len(o[0])
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            o = model.generate(
                **inputs,
                max_new_tokens=max_tokens_thinking,
                eos_token_id=stop_token_ids,
                temperature=0.1,
            )
            assert len(o) == 1
            o = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, o)]
            generated_text = tokenizer.decode(o[0], skip_special_tokens=True)
            prompt += generated_text
            print("***Initial prompt with more thinking:***\n", prompt)
        
        ### Final answer ###
        stop_token_ids = tokenizer("<|im_end|>")["input_ids"]
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        o = model.generate(
          **inputs,
          max_new_tokens=1000,
          eos_token_id=stop_token_ids,
          temperature=0.1,
        )
        assert len(o) == 1
        o = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, o)]
        generated_text = tokenizer.decode(o[0], skip_special_tokens=True)
        
        print("***Final output:***\n", prompt + generated_text)

In [7]:
EFFICIENT_TRAINING = True

In [8]:
compute_dtype = getattr(torch, "float16")
quant_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype, # Run computations in 16-bit float
    bnb_4bit_use_double_quant=False, # Use True for additional memory saving
) if EFFICIENT_TRAINING else None

In [24]:
base_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config if EFFICIENT_TRAINING else None,
    device_map={"": 0}
)
if EFFICIENT_TRAINING:
    base_model.config.use_cache = False
    base_model.config.pretraining_tp = 1 # A value other than 1 uses a slower, more accurate linear layer
base_tok = get_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     "qwem7b_vllm",
#     quantization_config=quant_config if EFFICIENT_TRAINING else None,
#     device_map={"": 0}
# )
model = peft.PeftModel.from_pretrained(base_model, "qwem7b_4bit_s1")
tok = get_tokenizer("qwem7b_4bit_s1")

In [35]:
reason_on_prompt(model, tok, "How many r's are in raspberry?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many r's are in raspberry?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many r's are in raspberry?<|im_end|>
<|im_start|>assistant
<|im_start|>think
Let's break down the process of counting the number of 'r' letters in the word "raspberry".

1. **Identify the target letter:** The question asks for the count of the letter 'r'.

2. **Analyze the word structure:**  The word "raspberry" is a compound word made up of two parts: "rasp" and "berry". This doesn't change the number of 'r's we need to find.

3. **Locate the first occurrence of 'r':**  Start reading the word from the beginning.
    * r - Found! So, there's at least one 'r'.
4. **Continue searching for subsequent occurrences:**  Move

In [36]:
reason_on_prompt(base_model, base_tok, "How many r's are in raspberry?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many r's are in raspberry?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many r's are in raspberry?<|im_end|>
<|im_start|>assistant
<|im_start|>thinker
To find out how many 'r's' are in the word "raspberry," I will count each occurrence of the letter 'r' in the word.

Here is the word: **raspberry**

Let's count:
1. The first 'r' is at the beginning.
2. The second 'r' is in the middle.
3. The third 'r' is at the end.

So, there are 3 'r's in the word "raspberry."
***Initial prompt with more thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many r's are in raspberry?<|im_end|>
<|im_start|>assistant
<|im_start

In [37]:
reason_on_prompt(model, tok, "If A > B and B > C, what can we conclude about A and C?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
If A > B and B > C, what can we conclude about A and C?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
If A > B and B > C, what can we conclude about A and C?<|im_end|>
<|im_start|>assistant
<|im_start|>think
The problem asks for a conclusion about the relationship between A and C given two inequalities: A > B and B > C.

Let's analyze the given inequalities step by step.

Inequality 1: A > B
This means that A is greater than B.
We can think of this as A being to the right of B on a number line.

Inequality 2: B > C
This means that B is greater than C.
We can think of this as B being to the right of C on a number line.

Now, let's combine these two inequalities.
Since A > B and B > C, we can use th

In [ ]:
reason_on_prompt(base_model, base_tok, "Rearrange the letters in 'LISTEN' to form another related word.")

In [44]:
reason_on_prompt(model, tok, "Rearrange the letters in 'LISTEN' to form another related word.")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Rearrange the letters in 'LISTEN' to form another related word.<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Rearrange the letters in 'LISTEN' to form another related word.<|im_end|>
<|im_start|>assistant
<|im_start|>think
Let's break down the thought process for rearranging the letters in 'LISTEN' to form another related word.

1. **Identify the Goal:** The core objective is to rearrange the letters of 'LISTEN' into a different word that has a connection to the original word. This means we need to think about synonyms, antonyms, or words with similar meanings.

2. **Analyze the Letters:**  Break down the letters: L, I, S, T, E, N.
    * Notice the presence of vowels (I, E, A) and consonants (L, 

KeyboardInterrupt: 

In [46]:
reason_on_prompt(base_model, base_tok, "How many unique ways can you arrange the letters in the word 'ATTACK'?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'ATTACK'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'ATTACK'?<|im_end|>
<|im_start|>assistant
<|im_start|>thinker
To determine the number of unique ways to arrange the letters in the word "ATTACK," we need to consider that it contains repeated letters. The word "ATTACK" consists of 6 letters where:

- A appears 3 times,
- T appears 2 times,
- C appears 1 time.

The formula for permutations of a multiset (a set with repeated elements) is given by:

\[
\frac{n!}{n_1! \cdot n_2! \cdot \ldots \cdot n_k!}
\]

where \( n \) is the total number of items to arrange, and \( n_1, n_2

In [47]:
reason_on_prompt(model, tok, "How many unique ways can you arrange the letters in the word 'ATTACK'?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'ATTACK'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'ATTACK'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
The problem asks for the number of unique arrangements of the letters in the word 'ATTACK'.
The word 'ATTACK' has 6 letters.
The letters are A, A, A, T, C, K.

We have 3 identical A's and 2 distinct letters T and K.

The formula to find the number of unique permutations of a set of objects where some objects are identical is given by:

n! / (n1! * n2! * ... * nk!)

where n is the total number of objects, and n1, n2, ..., nk are the frequencies of the distinct obje

KeyboardInterrupt: 

In [48]:
reason_on_prompt(base_model, base_tok, "How many unique ways can you arrange the letters in the word 'RACECAR'?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'RACECAR'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'RACECAR'?<|im_end|>
<|im_start|>assistant
<|im_start|>thinker
To determine the number of unique ways to arrange the letters in the word "RACECAR," we need to consider that this word has repeated letters. The word "RACECAR" consists of 7 letters where:

- R appears 2 times,
- A appears 2 times,
- C appears 1 time,
- E appears 1 time,
- and the last letter, R, also appears 1 time.

The formula for permutations of a multiset (a set with repeated elements) is given by:

\[
\frac{n!}{n_1! \cdot n_2! \cdot \ldots \cdot n_k!}
\

In [49]:
reason_on_prompt(model, tok, "How many unique ways can you arrange the letters in the word 'RACECAR'?")

***Initial prompt:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'RACECAR'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
***Initial prompt with thinking:***
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How many unique ways can you arrange the letters in the word 'RACECAR'?<|im_end|>
<|im_start|>assistant
<|im_start|>think
The problem asks for the number of unique arrangements of the letters in the word 'RACECAR'.
The word 'RACECAR' has 8 letters.
The letters are R, A, C, E, C, A, R.

To find the number of unique arrangements, we need to calculate the total number of permutations of these 8 letters.
Since there are repeated letters (C and A), we need to divide by the factorial of the counts of each repeated letter.

The formula for the number of unique permutations of a multiset i

KeyboardInterrupt: 